# <center><b>Time series template</b></center>

## <b>Step 0:</b> Context
*Explain the problem you're trying to solve, the nature of the dataset, the goal of the model, and the impact of the solution.*

## <b>Step 1:</b> Imports and configurations

### <b>Libraries</b>
*Import all necessary Python libraries (e.g., numpy, pandas, scikit-learn, tensorflow/pytorch for deep learning).*

In [ ]:
#- Workflow
from sklearn.pipeline import Pipeline
import mlflow
# import mlflow.sklearn

#- Data manipulation
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests

#- Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

#- Preprocessing for model fitting
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.stattools import adfuller

#- ML models
import optuna
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import PoissonRegressor


#- Model assessment for regression
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
)
from sklearn.model_selection import cross_val_score, KFold
from scipy import stats

### <b>Configurations</b>
*Set any global configurations such as random seed for reproducibility, backend settings for computation libraries, and any project-specific parameters.*

To keep track of our experiments we have used mlflow. You'll need to set up an ML flow server to run this notebook to the end. For that follow the instructions

1️⃣ Install MLflow
Run the following command in your terminal to install MLflow:

pip install mlflow

2️⃣ Start the MLflow Tracking Server
In your terminal, launch the MLflow UI by running:

mlflow ui

    The UI will be available at http://localhost:5000.

3️⃣ Visualize Results
Open http://localhost:5000 in your browser to track metrics, parameters, and model artifacts for all our experiments.

 Optional: You can specify a different tracking server in the flollowing code cell using:

mlflow.set_tracking_uri("http://your-server-address:5000")

In [ ]:
SEED = 42

#- Show all the column
pd.set_option('display.max_columns', None)

#- Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("test_mlflow_setup")      # This is just to test if the mlflow server is working
try:
    mlflow.log_param("test_param", "test_value")
    print("MLflow is configured correctly!")
except Exception as e:
    print(f"MLflow configuration failed: {e}")

### <b>Dataset</b>
*If you have several data sets, you need to duplicate the cells below. If your dataset is in a different format, create a new cell and write the code to import it. Don't forget Chat GPT and Google are your friends. ;-)*

*If your data is in a CSV file use the cell below to import it.*

In [ ]:
data_path = ".csv"
data = pd.read_csv(data_path)
raw_data = data.copy()
data.head(3)

*If your data is in an excel file use the cell below to import it instead.*

In [ ]:
# data_path = ".xlsx"
# data = pd.read_excel(data_path)
# raw_data = data.copy()
# data.head(3)

## <b>Step 2:</b> Data inspection and cleanings

### <b>Inspection</b>
*The goal of data inspection is to understand the dataset's characteristics, quality, and any potential issues that need addressing before analysis.*

In [ ]:
def basic_inspection(df: pd.DataFrame, plot_missing_data=True, detect_duplicates=True, include_statistics=True):
    """
        Perform a comprehensive inspection of a pandas DataFrame, including an overview of its size,
        data types, presence of null values, detection of duplicate rows, and optional detailed statistics.

        Parameters:
        - df (pandas.DataFrame): The DataFrame to inspect.
        - plot_missing_data (bool): If True, generates a heatmap of missing values. Default is True.
        - detect_duplicates (bool): If True, checks for and reports the number of duplicate rows. Default is True.
        - include_statistics (bool): If True, includes detailed summary statistics for numeric columns and 
        the most common values for categorical columns in the returned summary table. Default is True.

        Returns:
        - pandas.DataFrame: A summary table that includes:
            - Unique_values: Number of unique values in each column.
            - Data_type: The data type of each column.
            - Null_count: The number of missing values in each column.
            - Null_percentage: The percentage of missing values in each column.
            - (Optional) Most_common: The most frequent value in each categorical column.
            - (Optional) Mean, Std, Min, Max: Basic statistics for numeric columns.

        Prints:
        - Basic DataFrame information, including the number of rows, columns, and data types.
        - Columns with missing values and their respective counts.
        - Heatmap of missing values if plot_missing_data is True.
        - Nullity correlation heatmap if multiple columns contain missing values.
        - Number of duplicate rows if detect_duplicates is True.
    """
    #- Basic informations
    print(f"Number of rows: {df.shape[0]}\nNumber of columns: {df.shape[1]}\n")
    print("Data types:", df.dtypes, sep='\n')

    #- Missing values
    missing_value_cnt = df.isnull().sum()
    num_cols_with_missing = len(missing_value_cnt[missing_value_cnt > 0])
    if num_cols_with_missing == 0:
        print(f"\n{'-'*100}\nThere are no null values in the dataset.\n{'-'*100}")
        if plot_missing_data:
            plt.figure(figsize=(15, 7))
            sns.heatmap(df.isna(), cbar=True, cmap='viridis')
            plt.title('Heatmap of missing values')
            plt.show()
    else:
        print(f"\n{'-'*100}\nColumns with null values:\n", missing_value_cnt[missing_value_cnt > 0])
        if plot_missing_data:
            plt.figure(figsize=(15, 7))
            sns.heatmap(df.isna(), cbar=True, cmap='viridis')
            plt.title('Heatmap of missing values')
            plt.show()

            if num_cols_with_missing > 1:
                missing_value_df = df.iloc[:, [i for i, n in enumerate(np.var(df.isnull(), axis='rows')) if n > 0]]
                missing_corr_mat = missing_value_df.isnull().corr()
                print(
                    """
                        The correlation heatmap is based on the missingno correlation heatmap. It measures 
                        nullity correlation: how strongly the presence or absence of one variable affects the 
                        presence of another. Nullity correlation ranges from -1 (if one variable appears the 
                        other definitely does not) to 0 (variables appearing or not appearing have no effect 
                        on one another) to 1 (if one variable appears the other definitely also does).
                    """
                )
                plt.figure(figsize=(15, 10))
                sns.heatmap(missing_corr_mat, annot=True, cmap='coolwarm')
                plt.title("Nullity correlation heatmap")
                plt.show()

    #- Duplicated rows
    if detect_duplicates:
        duplicated_rows_cnt = df.duplicated().sum()
        if duplicated_rows_cnt > 0:
            print(f"\n{''*100}\nNumber of duplicated rows: {duplicated_rows_cnt}\n{''*100}")
        else:
            print(f"\n{''*100}\nThere are no duplicated rows in the dataset.\n{''*100}")

    #- Summary table
    summary_table = pd.DataFrame({
        "Unique_values": df.nunique(),
        "Data_type": df.dtypes,
        "Null_count": df.isnull().sum(),
        "Null_percentage": (df.isnull().sum() / df.shape[0] * 100).round(2)
    })

    if include_statistics:
        # Most common values in categorical columns
        summary_table['Most_common'] = df.select_dtypes(include=['object']).apply(
            lambda x: x.mode()[0] if len(x.mode()) > 0 else 'N/A')

        # Summary statistics for numeric columns
        for col in df.select_dtypes(include=['int64', 'float64']).columns:
            summary_table.loc[col, 'Mean'] = df[col].mean()
            summary_table.loc[col, 'Std'] = df[col].std()
            summary_table.loc[col, 'Min'] = df[col].min()
            summary_table.loc[col, 'Max'] = df[col].max()
    
    return summary_table        # I use a return here for a better visualization. You can replace this line with: "print(summary_table)"

*For the following cell, modify the parameters according to the particularities of your database. For example, if you have intentional duplicates, it is not necessary to check them.*

In [ ]:
basic_inspection(data, visualize_nulls=True, detailed_summary=True, check_duplicates=True)

In [ ]:
def identify_outliers(df, plot=True):
    """
    Identify outliers in a pandas DataFrame using Z-score (values more than 3 standard deviations from 
    the mean) and Interquartile Range (IQR) methods (values below Q1 - 1.92*IQR or above Q3 + 1.92*IQR).
    Outliers identified by both methods are combined to provide a comprehensive overview of outliers 
    in each numeric feature.
    
    Parameters:
    - df (pd.DataFrame): A pandas DataFrame containing the data to be analyzed. Only numeric
      columns will be considered for outlier detection.
    
    Returns:
    - dict: A dictionary where each key is the name of a numeric feature in the DataFrame. Each value
      is another dictionary containing two keys: 'num_outliers', which is the number of unique outliers
      identified in the feature, and 'outliers_index', an array of indices of these outliers.
    """
    outlier_summary = {}
    
    for column in df.select_dtypes(include=np.number).columns:  # Focus on numeric columns
        # Calculate Z-scores
        z_scores = np.abs(stats.zscore(df[column].dropna()))
        z_outliers = np.where(z_scores > 3)[0]
        
        # Calculate IQR
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        iqr_outliers = df[(df[column] < (Q1 - 1.96 * IQR)) | (df[column] > (Q3 + 1.96 * IQR))].index
        
        # Combine unique outliers from both methods
        combined_outliers = np.union1d(z_outliers, iqr_outliers)
        
        # Optionally plot
        if plot:
            print(column)
            plt.figure(figsize=(8, 4))
            sns.boxplot(x=df[column])
            plt.title(f'Boxplot of {column} (Outliers highlighted)')
            plt.show()

        # Summary
        outlier_summary[column] = {
            'num_outliers': len(combined_outliers),
            'outliers_index': combined_outliers
        }
    
    return outlier_summary

*This function does not modify the original DataFrame. I recommend you to inspect the identified outliers and decide on appropriate handling methods such as removal, replacement, or keeping them as is, depending on the analysis requirements and domain knowledge.*

In [ ]:
outlier_info = identify_outliers(data)
print(outlier_info)

### <b> Cleaning</b>
*Following the inspection, data cleaning aims to rectify issues identified, improving the dataset's quality and making it suitable for further analysis and modeling.*

In [ ]:
data.head(3)

In [ ]:
#- Feature Selection

# data.drop(column=[], inplace=True)

In [ ]:
#- Remove duplicates

# data.drop_duplicates(inplace=True)

You need to adapt the following function to your own data. You can add or remove transformations, or modify the way they work.

In [ ]:
def data_cleaning(data, date_col='date_column'):
    """
    Clean and preprocess data for time series prediction.

    Args:
        data (pd.DataFrame): The input data to be cleaned.
        date_col (str): The column containing datetime information.

    Returns:
        pd.DataFrame: The cleaned and preprocessed data.
    """
    #- Ensure the date column is in datetime format
    data[date_col] = pd.to_datetime(data[date_col], errors='coerce')
    
    #- Sort data by the date column
    data = data.sort_values(by=date_col).reset_index(drop=True)
    
    #- Handle missing values with forward and backward fill (common for time series)
    for col in data.select_dtypes(include=[np.number]).columns:
        data[col].fillna(method='ffill', inplace=True)
        data[col].fillna(method='bfill', inplace=True)
    
    #- Interpolation for smoother time series (optional)
    for col in data.select_dtypes(include=[np.number]).columns:
        data[col] = data[col].interpolate(method='linear')

    #- Extract time-based features
    data['year'] = data[date_col].dt.year
    data['month'] = data[date_col].dt.month
    data['day'] = data[date_col].dt.day
    data['day_of_week'] = data[date_col].dt.dayofweek
    data['quarter'] = data[date_col].dt.quarter
    data['is_weekend'] = data[date_col].dt.dayofweek >= 5
    data['hour'] = data[date_col].dt.hour if data[date_col].dt.hour.nunique() > 1 else 0
    
    #- Generate lag features (useful for time series forecasting)
    lag_features = ['numerical_column']  # Replace with relevant numerical columns
    for col in lag_features:
        for lag in range(1, 4):  # Create lag features for the last 3 time steps
            data[f'{col}_lag_{lag}'] = data[col].shift(lag)
    
    #- Rolling window features (moving averages, rolling sum, etc.)
    for col in lag_features:
        data[f'{col}_rolling_mean_3'] = data[col].rolling(window=3).mean()
        data[f'{col}_rolling_std_3'] = data[col].rolling(window=3).std()

    #- Drop rows with remaining missing values (after creating lags and rolling features)
    data.dropna(inplace=True)

    #- Encoding categorical variables
    categorical_cols = ['categorical_column']  # Replace with actual categorical columns
    for col in categorical_cols:
        data[col] = data[col].astype('category').cat.codes

    #- One-hot encoding (optional)
    data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
    
    return data


In [ ]:
data = data_cleaning(data, date_col)        # You'll have to identify the column of the dataframe that contain the date.

<i>When deciding between one-hot encoding and simple encoding (label encoding) for categorical data, consider the nature of your categories:

- One-Hot Encoding: Use this when your categorical variable does not have a meaningful order or hierarchy. One-hot encoding creates a new binary column for each category, which is ideal for nominal data (e.g., color, city names). This approach prevents the model from assuming a natural ordering between categories, which is helpful to avoid misleading interpretations.

- Label Encoding: Use this when your categorical variable has a meaningful order or ranking (ordinal data). Label encoding assigns a unique integer to each category, preserving the order. However, be cautious, as this can introduce unintended ordinal relationships that may not exist in the data.</i>

In [ ]:
#- Handle missing values
data['numerical_column'].fillna(data['numerical_column'].mean(), inplace=True)
data['categorical_column'].fillna(data['categorical_column'].mode()[0], inplace=True)

#- Correct data type
data['numeric_column_as_string'] = pd.to_numeric(data['numeric_column_as_string'], errors='coerce')
data['date_column'] = pd.to_datetime(data['date_column'])

#- Binning
#- Fixed width binning
data['value_bin'] = pd.cut(data['column_to_bin'], bins=3, labels=["Low", "Medium", "High"])
#- Qauntile based binning
data['value_quantile_bin'] = pd.qcut(data['column_to_bin'], q=3, labels=["Low", "Medium", "High"])
#- Custom binning
bins = []
data['value_custom_bin'] = pd.cut(data['column_to_bin'], bins=bins, labels=["Low", "Medium", "High"], right=False)

#- Turn categorical variables into numerical variables
#- Label encoding
# Define a mapping of categories to numerical values
checking_mapping = {'None': 0, 'little': 1, 'moderate': 2, 'rich': 3}
# Map the categories to their numerical equivalents
data['col_name'] = data['col_name_numb'].map(checking_mapping)
#- One-hot encoding of categorical variables
categorical_cols = []       # Add the columns of interest
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

In [ ]:
#- Outlier Treatment

# for col, info in outlier_info.items():
#     data = data.drop(index=info['outliers_index'], inplace=False)

In [ ]:
data.head(3)

## <b>Step 3:</b> Exploratory Data Analysis (problem navigation)

In [ ]:
def visualize_time_series(data, time_col, target_col, freq='D', title=None):
    """
    Visualize the target feature over time with flexible time aggregation, including hourly data.

    Args:
        data (pd.DataFrame): The input data containing time and target columns.
        time_col (str): The column name representing time (must be in datetime format).
        target_col (str): The column name representing the target variable to visualize.
        freq (str): Frequency for resampling. Examples:
            - 'H': Hourly
            - 'D': Daily
            - 'W': Weekly
            - 'M': Monthly
            - 'Q': Quarterly
            - 'Y': Yearly
        title (str): Optional title for the plot.
    
    Returns:
        None: Displays a plot of the target feature over time.
    """
    # Ensure the time column is in datetime format
    data[time_col] = pd.to_datetime(data[time_col], errors='coerce')
    
    # Remove rows with missing values in the time or target columns
    data = data.dropna(subset=[time_col, target_col])
    
    # Set the time column as the index
    data = data.set_index(time_col)
    
    # Resample the data based on the specified frequency
    resampled_data = data[target_col].resample(freq).mean()
    
    # Plot the resampled time series
    plt.figure(figsize=(14, 7))
    plt.plot(resampled_data.index, resampled_data.values, marker='o', linestyle='-', markersize=4)
    plt.xlabel('Time')
    plt.ylabel(target_col)
    plt.title(title if title else f'{target_col} Over Time ({freq} Frequency)')
    plt.grid(True)
    
    # Improve x-axis ticks for large datasets or sub-daily frequency
    if freq in ['H', 'D', 'W']:
        plt.gcf().autofmt_xdate()
    
    plt.show()


In [ ]:
# Visualize hourly data
visualize_time_series(data, date_col, target_col=target_col, freq='H')      # You'll have to specify the column that contains the time/date and the column that contains the target feature.

In [ ]:
# Get unique categories
unique_categories = data[category_col].unique()

# Loop over each unique category
for category in unique_categories:
    print(f"\Display for {category_col}: {category}")
    
    # Filter the data for the current category
    category_data = data[data[category_col] == category].copy()
    
    visualize_time_series(category_data, date_col, target_col=target_col, freq='H')  

To display other timeframe, change the frequency.

In [ ]:
def time_based_heatmap(data, time_col, value_col, x_freq, y_freq, agg_func='sum', cmap='coolwarm'):
    """
    Generate a heatmap for time-based analysis with customizable resolutions (e.g., hour/day, week/day, month/week).

    Args:
        data (pd.DataFrame): The input data containing time and value columns.
        time_col (str): The column representing time (must be in datetime format).
        value_col (str): The column with values to aggregate (e.g., 'Entrées').
        x_freq (str): Frequency for the x-axis. Options: 'hour', 'day', 'week', 'month', 'year'.
        y_freq (str): Frequency for the y-axis. Options: 'hour', 'day', 'week', 'month', 'year'.
        agg_func (str): Aggregation function to apply ('sum', 'mean', etc.).
        cmap (str): Color map for the heatmap.

    Returns:
        None: Displays a heatmap.
    """
    # Ensure the time column is in datetime format
    data[time_col] = pd.to_datetime(data[time_col], errors='coerce')
    
    # Extract the necessary time-based features
    time_features = {
        'hour': data[time_col].dt.hour,
        'day': data[time_col].dt.day_name(),
        'week': data[time_col].dt.isocalendar().week,
        'month': data[time_col].dt.month_name(),
        'year': data[time_col].dt.year
    }
    
    # Ensure x_freq and y_freq are valid
    if x_freq not in time_features or y_freq not in time_features:
        raise ValueError(f"Invalid frequency. Choose from {list(time_features.keys())}")
    
    # Add the selected features to the DataFrame
    data['x_feature'] = time_features[x_freq]
    data['y_feature'] = time_features[y_freq]
    
    # Create a pivot table for the heatmap
    pivot_table = data.pivot_table(index='y_feature', columns='x_feature', values=value_col, aggfunc=agg_func)
    
    # Plot the heatmap
    plt.figure(figsize=(14, 7))
    sns.heatmap(pivot_table, cmap=cmap, linewidths=0.5, annot=True, fmt=".0f")
    plt.title(f"Heatmap of {value_col} by {y_freq.capitalize()} and {x_freq.capitalize()}")
    plt.xlabel(x_freq.capitalize())
    plt.ylabel(y_freq.capitalize())
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Month/Year heatmap
time_based_heatmap(data, 
                  time_col=date_col, 
                  value_col=target_col, 
                  x_freq='year', 
                  y_freq='month', 
                  agg_func='sum')

In [ ]:
# Get unique categories
unique_categories = data[category_col].unique()

# Loop over each unique category
for category in unique_categories:
    print(f"\Display for {category_col}: {category}")
    
    # Filter the data for the current category
    category_data = data[data[category_col] == category].copy()
    
    # Month/Year heatmap
    time_based_heatmap(category_data, 
                  time_col=date_col, 
                  value_col=target_col, 
                  x_freq='year', 
                  y_freq='month', 
                  agg_func='sum')

In [ ]:
def quant_univariate_analysis(df, col_name, visualize=True, **kwargs):
    """
    Perform univariate analysis on a numerical column with visualization
    options using a Matplotlib native color palette focusing on red, yellow, and black.

    Parameters:
    df (DataFrame): Input DataFrame containing the data.
    col_name (str): Name of the numerical column to analyze.
    visualize (bool): Whether to visualize the analysis (default=True).
    **kwargs: Additional keyword arguments to customize the visualisation colors.

    Returns:
    pandas.Series: Descriptive statistics of the column.
    """
    if col_name not in df.columns:
        raise ValueError(f"Column '{col_name}' not found in the DataFrame.")

    # 1. Descriptive Statistics
    descriptive_stats = df[col_name].describe()
    print("\n"*2, col_name, "\n"*2, descriptive_stats)

    if visualize:
        # Custom color palette
        custom_colors = ['red', 'yellow', 'black']  # Red, Yellow, Black
        color = kwargs.get('hist_color', custom_colors[0])  # Use red as default
        
        # 2. Histogram with KDE
        plt.figure(figsize=kwargs.get('figsize', (12, 6)))
        sns.histplot(df[col_name], kde=True, bins=kwargs.get('bins', 30),
                     color=color,
                     kde_kws={'bw_adjust': kwargs.get('bw_adjust', 1)},
                     line_kws={'color': custom_colors[2], 'lw': 2})  # Use black for KDE line
        plt.title(f'Histogram and KDE of {col_name}')
        plt.xlabel(col_name)
        plt.ylabel('')
        plt.grid(True, linestyle='--', linewidth=0.5, color=custom_colors[1])  # Use yellow for grid lines
        plt.show()

        # 3. Boxplot
        plt.figure(figsize=kwargs.get('figsize', (12, 6)))
        sns.boxplot(x=df[col_name], color=kwargs.get('boxplot_color', custom_colors[1]))  # Use yellow for boxplot
        plt.title(f'Boxplot of {col_name}')
        plt.xlabel(col_name)
        plt.grid(True, linestyle='--', linewidth=0.5, color=custom_colors[2])  # Use black for grid lines
        plt.show()

    return descriptive_stats

In [ ]:
for quant_col in []:        # Add in the brackets the name of the quantitative variables in your dataset that you want to visualize.
    quant_univariate_analysis(data, quant_col)

In [ ]:
def qual_univariate_analysis(df, col_name, palette="viridis", show_grid=True, figsize=(12, 6)):
    """
    Performs and visualizes a univariate analysis for a qualitative (categorical) variable,
    highlighting and annotating the most and least common categories.

    Parameters:
    - df (DataFrame): The pandas DataFrame containing the data.
    - col_name (str): The name of the column to analyze.
    - palette (str, optional): Color palette for the plots. Defaults to 'viridis'.
    - show_grid (bool, optional): Whether to show the grid in the bar plot. Defaults to True.
    - figsize (tuple, optional): Figure size for the plots. Defaults to (12, 6).
    """
    
    # Frequency Table
    freq_table = df[col_name].value_counts()
    # Percentage Table
    percent_table = df[col_name].value_counts(normalize=True) * 100
    combined_table = pd.DataFrame({'Frequency': freq_table, 'Percentage': percent_table})
    print("\n"*2, col_name, "\n"*2, combined_table)

    # Identify most and least common categories
    most_common = freq_table.idxmax()
    least_common = freq_table.idxmin()

    # Bar Plot with Highlighting
    plt.figure(figsize=figsize)
    barplot = sns.countplot(y=df[col_name], order=freq_table.index, palette=palette)
    
    # Highlighting
    for patch in barplot.patches:
        if patch.get_y() == freq_table.index.get_loc(most_common):
            patch.set_facecolor('green')  # Highlight most common category in green
        elif patch.get_y() == freq_table.index.get_loc(least_common):
            patch.set_facecolor('red')  # Highlight least common category in red
    
    # Annotations
    plt.title(f'Bar Plot of {col_name}')
    plt.xlabel('Count')
    plt.ylabel(col_name)
    if show_grid:
        plt.grid(axis='x', linestyle='--', linewidth=0.5)
    
    # Adding annotations for the most and least common categories
    plt.text(freq_table.max(), freq_table.index.get_loc(most_common), 'Most common', fontsize=12, va='center')
    plt.text(freq_table.min(), freq_table.index.get_loc(least_common), 'Least common', fontsize=12, va='center')
    
    plt.show()

    return combined_table

In [ ]:
for qual_col in []:         # Add in the brackets the name of the qualitative variables in your dataset that you want to visualize.
    qual_univariate_analysis(data, qual_col)

In [ ]:
def visualize_multivariate_time_series(data, time_col, target_col, condition_col, condition_func, freq='D', title=None):
    """
    Visualize the target time series with highlighted points where another variable meets a certain condition.

    Args:
        data (pd.DataFrame): The input data containing time, target, and condition columns.
        time_col (str): The column name representing time (must be in datetime format).
        target_col (str): The column name representing the target variable to visualize.
        condition_col (str): The column name representing the variable to check the condition.
        condition_func (function): A function that takes a pandas Series and returns a boolean Series indicating the condition.
        freq (str): Frequency for resampling (e.g., 'H' for hourly, 'D' for daily, 'M' for monthly).
        title (str): Optional title for the plot.

    Returns:
        None: Displays a plot of the target feature over time with highlights.
    """
    # Ensure the time column is in datetime format
    data[time_col] = pd.to_datetime(data[time_col], errors='coerce')
    
    # Remove rows with missing values in the time or target columns
    data = data.dropna(subset=[time_col, target_col, condition_col])
    
    # Set the time column as the index
    data = data.set_index(time_col)
    
    # Resample the data based on the specified frequency
    resampled_data = data[[target_col, condition_col]].resample(freq).mean()
    
    # Apply the condition function to determine where to highlight
    condition_met = condition_func(resampled_data[condition_col])
    
    # Plot the target time series
    plt.figure(figsize=(14, 7))
    plt.plot(resampled_data.index, resampled_data[target_col], label=target_col, color='blue', linestyle='-')
    
    # Highlight points where the condition is met
    plt.scatter(resampled_data.index[condition_met], resampled_data[target_col][condition_met], 
                color='red', label=f'{condition_col} Condition Met', zorder=5)
    
    # Plot settings
    plt.xlabel('Time')
    plt.ylabel(target_col)
    plt.title(title if title else f'{target_col} Over Time with {condition_col} Highlights')
    plt.legend()
    plt.grid(True)
    plt.gcf().autofmt_xdate()
    plt.show()


Define a condition for highlighting using a condition function (condition_func), which should take a Pandas Series and return a boolean mask where the condition is met. For example, to highlight where a column temperature exceeds 30, use lambda x: x > 30. Call the function with the appropriate parameters and specify the frequency (freq) for resampling (H for hourly, D for daily, etc.). The function will plot the target variable over time and highlight points where the condition is true, making it easy to spot patterns, relations and anomalies in your time series data.

In [ ]:
#- Define the condition function: temperature > 30
condition_func = lambda x: x > 30

#- Visualize the time series with highlights
visualize_multivariate_time_series(data, time_col=date_col, target_col=target_col, 
                                   condition_col=condition_col, condition_func=condition_func, freq='D')


In [ ]:
# Get unique categories
unique_categories = data[category_col].unique()

# Loop over each unique category
for category in unique_categories:
    print(f"\Display for {category_col}: {category}")
    
    # Filter the data for the current category
    category_data = data[data[category_col] == category].copy()
    
    #- Visualize the time series with highlights
    visualize_multivariate_time_series(category_data, time_col=date_col, target_col=target_col, 
                                   condition_col=condition_col, condition_func=condition_func, freq='D')


### <b>Auto-correlation analysis</b>

In [ ]:
def plot_acf_pacf(series, lags=50, title_prefix="Time Series"):
    """
    Plot both ACF and PACF for a given time series.

    Args:
        series (pd.Series): The time series data.
        lags (int): The number of lags to display on the ACF and PACF plots.
        title_prefix (str): Prefix for the plot titles.
        
    Returns:
        None: Displays the ACF and PACF plots.
    """
    plt.figure(figsize=(15, 8))
    
    # Plot ACF
    plt.subplot(2, 1, 1)
    plot_acf(series.dropna(), lags=lags, ax=plt.gca())
    plt.title(f'{title_prefix} - Autocorrelation (ACF)')
    
    # Plot PACF
    plt.subplot(2, 1, 2)
    plot_pacf(series.dropna(), lags=lags, ax=plt.gca())
    plt.title(f'{title_prefix} - Partial Autocorrelation (PACF)')
    
    plt.tight_layout()
    plt.show()


In [ ]:
# Example: Plot ACF and PACF for a time series
plot_acf_pacf(data[target_col], lags=52, title_prefix="Mall Entries")

In [ ]:
# Get unique categories
unique_categories = data[category_col].unique()

# Loop over each unique category
for category in unique_categories:
    print(f"\Display for {category_col}: {category}")
    
    # Filter the data for the current category
    category_data = data[data[category_col] == category].copy()
    
    # Example: Plot ACF and PACF for a time series
    plot_acf_pacf(category_data[target_col], lags=52, title_prefix="Mall Entries")


In [ ]:
def adf_test(series):
    """
    Performs the Augmented Dickey-Fuller test on a given time series.
    
    Parameters:
    -----------
    series : pd.Series
        The time series to test.
        
    Returns:
    --------
    dict
        A dictionary containing the Mall_ID, ADF statistic, p-value, and stationarity status.
    """
    result = adfuller(series.dropna())
    return {
        "ADF Statistic": result[0],
        "p-value": result[1],
        "Critical Values": result[4],
        "Stationary": result[1] < 0.05
    }

In [ ]:
adf_test(data[target_col])

In [ ]:
# Get unique categories
unique_categories = data[category_col].unique()

# Loop over each unique category
for category in unique_categories:
    print(f"\Display for {category_col}: {category}")
    
    # Filter the data for the current category
    category_data = data[data[category_col] == category].copy()
    
    adf_test(category_data)

### <b>Correlation</b>

In [ ]:
#- Calculate the correlation matrix
correlation_cols = []       # Add in the brackets the name of the the columns you want to visualize for correlation. Make sure these columns are numeric. You can numerize the qualitative columns back in the preprocessing step.
correlation = data[correlation_cols].corr(method='pearson')

#- Visualize the correlation matrix
fig, ax = plt.subplots()

ax.figure.set_size_inches(10, 10)
mask = np.triu(np.ones_like(correlation, dtype=bool))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(correlation, cmap=cmap, mask=mask, square=True, linewidths=.5, 
            annot=True, annot_kws={'size':14})

plt.show()

In [ ]:
strong_corr = correlation[(correlation > 0.7) | (correlation < -0.7)]
print("Strong correlations:\n", strong_corr)

## <b>Step 4:</b> Feature Engineering
*The Feature engineering phase involves transforming raw data into meaningful features that effectively represent the underlying problem.*  
*<b>Feature Creation:</b>* Develop new features from the existing data to better capture the underlying patterns.  
*<b>Feature Transformation:</b>* Apply transformations (e.g., scaling, encoding) to make the data suitable for modeling.  
*<b>Feature Selection:</b>* Use statistical tests and selection algorithms to reduce dimensionality and focus on relevant features.

In [ ]:
def do_feature_eng(df):
    return df

In [ ]:
data = do_feature_eng(data)

## <b>Step 5:</b> Data preparation

### <b>Train test split</b>

In [ ]:
# Ccutoff dates for splitting
tt_split_date = "2024-10-01"        # All data after this date will be used for test
tv_split_date = "2024-07-01"        # All data between this date and tt_split_date will be used for validation

In [ ]:
# Train set
train_df = data[data[date_col] < tv_split_date]
X_train, y_train = train_df.drop(columns=[target_col]), train_df[target_col]

# Validation set
val_df = data[(data[date_col] >= tv_split_date) & (data[date_col] < tt_split_date)]
X_val, y_val = val_df.drop(columns=[target_col]), val_df[target_col]

# Test set
test_df = data[data[date_col] >= tt_split_date]
X_test, y_test = test_df.drop(columns=[target_col]), test_df[target_col]

full_train_df = data[data[date_col] < tt_split_date]
X_full_train, y_full_train = full_train_df.drop(columns=[target_col]), full_train_df[target_col]

print(f"Train size: {X_train.shape}, Validation size: {X_val.shape}, Test size: {X_test.shape}, Full train size: {X_full_train.shape}")

## <b>Step 6:</b> Training & evaluation

### <b>Fitting a base model</b>

In [ ]:
#- Start a new MLflow run
mlflow.set_experiment("baseline_model")

In [ ]:
with mlflow.start_run():
    #- Define the baseline model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())  # Use Linear Regression as the model
    ])

    # Fit the pipeline to training data
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_val)
    
    # Calculate evaluation metrics for regression
    mse = mean_squared_error(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)

    print(f"\nMean Squared Error (MSE): {mse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R² Score: {r2:.4f}")
    
    # Log metrics with MLflow
    mlflow.log_metric('mean_squared_error', mse)
    mlflow.log_metric('mean_absolute_error', mae)
    mlflow.log_metric('r2_score', r2)
    
    # Residuals plot
    residuals = y_val - y_pred
    plt.figure(figsize=(10, 6))
    sns.residplot(x=y_val, y=residuals, lowess=True, color="blue")
    plt.axhline(0, linestyle="--", color="red")
    plt.title("Residuals Plot")
    plt.xlabel("True Values")
    plt.ylabel("Residuals")
    plt.grid(True)
    plt.savefig("residuals_plot.png")
    plt.show()
    
    # Log Residuals Plot as artifact
    mlflow.log_artifact("residuals_plot.png")
    
    # Predicted vs True values plot
    plt.figure(figsize=(10, 6))
    plt.scatter(y_val, y_pred, alpha=0.7, color="green")
    plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], '--r', linewidth=2)
    plt.title("Predicted vs True Values")
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.grid(True)
    plt.savefig("predicted_vs_true.png")
    plt.show()
    
    # Log Predicted vs True Plot as artifact
    mlflow.log_artifact("predicted_vs_true.png")
    
    # Save the model and log it with MLflow
    mlflow.sklearn.log_model(pipeline, "baseline_model")


### <b>Fitting an hyper-optimized model</b>

In [ ]:
#- Start a new MLflow run
mlflow.set_experiment("optimized_model")

In [ ]:
#- Define regressors and their parameter grids
regressors = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "ElasticNet": ElasticNet(),
    "Polynomial Regression": Pipeline([
        ("poly_features", PolynomialFeatures(include_bias=False)),
        ("regressor", LinearRegression())
    ]),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBoost Regressor": XGBRegressor(),
    "Support Vector Regressor": SVR(),
    "K-Nearest Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Gradient Boosting Regressor": GradientBoostingRegressor()
}

In [ ]:
def optuna_mlflow_tuner(trial):
    """
    Function to optimize hyperparameters of a wide range of regressors using Optuna 
    and log detailed results with MLflow. Combines advanced feature engineering, 
    standardized scaling, and regression evaluation with detailed metrics and visualizations.

    Parameters:
    -----------
    trial : optuna.trial.Trial
        The trial object provided by Optuna to suggest hyperparameters and track the results.

    Returns:
    --------
    float
        The main evaluation metric (Mean Squared Error) of the model on the validation set.
    """
    global best_model

    # Start an MLflow run for this trial
    with mlflow.start_run():
        # Select a regressor
        model_name = trial.suggest_categorical("model", list(regressors.keys()))
        regressor = regressors[model_name]

        # Suggest hyperparameters specific to the selected regressor
        if model_name == "Linear Regression":
            fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
            regressor.set_params(fit_intercept=fit_intercept)

        elif model_name == "Ridge Regression":
            alpha = trial.suggest_loguniform("ridge_alpha", 0.001, 100)
            regressor.set_params(alpha=alpha)

        elif model_name == "Lasso Regression":
            alpha = trial.suggest_loguniform("lasso_alpha", 0.001, 100)
            regressor.set_params(alpha=alpha)

        elif model_name == "ElasticNet":
            alpha = trial.suggest_loguniform("elasticnet_alpha", 0.001, 100)
            l1_ratio = trial.suggest_uniform("elasticnet_l1_ratio", 0.1, 1.0)
            regressor.set_params(alpha=alpha, l1_ratio=l1_ratio)

        elif model_name == "Polynomial Regression":
            degree = trial.suggest_int("polynomial_degree", 2, 5)
            regressor.set_params(poly_features__degree=degree)

        elif model_name == "Random Forest Regressor":
            n_estimators = trial.suggest_int("n_estimators", 50, 500, step=50)
            max_depth = trial.suggest_int("max_depth", 3, 20)
            min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
            min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
            regressor.set_params(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                n_jobs=-1
            )

        elif model_name == "XGBoost Regressor":
            max_depth = trial.suggest_int("max_depth", 3, 15)
            learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 0.5)
            n_estimators = trial.suggest_int("n_estimators", 50, 500, step=50)
            subsample = trial.suggest_uniform("subsample", 0.5, 1.0)
            colsample_bytree = trial.suggest_uniform("colsample_bytree", 0.5, 1.0)
            regressor.set_params(
                max_depth=max_depth,
                learning_rate=learning_rate,
                n_estimators=n_estimators,
                subsample=subsample,
                colsample_bytree=colsample_bytree
            )

        elif model_name == "Support Vector Regressor":
            kernel = trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])
            C = trial.suggest_loguniform("C", 0.1, 100)
            epsilon = trial.suggest_loguniform("epsilon", 0.01, 1.0)
            regressor.set_params(kernel=kernel, C=C, epsilon=epsilon)

        elif model_name == "K-Nearest Neighbors Regressor":
            n_neighbors = trial.suggest_int("n_neighbors", 1, 50)
            weights = trial.suggest_categorical("weights", ["uniform", "distance"])
            algorithm = trial.suggest_categorical("algorithm", ["auto", "ball_tree", "kd_tree", "brute"])
            regressor.set_params(n_neighbors=n_neighbors, weights=weights, algorithm=algorithm)

        elif model_name == "Decision Tree Regressor":
            max_depth = trial.suggest_int("max_depth", 3, 20)
            min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
            min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
            regressor.set_params(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

        elif model_name == "Gradient Boosting Regressor":
            learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 0.5)
            n_estimators = trial.suggest_int("n_estimators", 50, 500, step=50)
            max_depth = trial.suggest_int("max_depth", 3, 15)
            regressor.set_params(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)

        # Build the pipeline
        pipeline = Pipeline([
            ("feature_engineering", FunctionTransformer(do_feature_eng, validate=False)),  # Replace with your feature engineering function
            ("scaler", StandardScaler()),
            ("regressor", regressor)
        ])

        # Train the pipeline
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_val)

        # Calculate evaluation metrics for regression
        mse = mean_squared_error(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)

        # Log metrics in MLflow
        mlflow.log_metric("mean_squared_error", mse)
        mlflow.log_metric("mean_absolute_error", mae)
        mlflow.log_metric("r2_score", r2)
        mlflow.log_param("model", model_name)

        # Log hyperparameters
        params = regressor.get_params()
        for param_name, param_value in params.items():
            mlflow.log_param(param_name, param_value)

        # Plot predicted vs actual values
        plt.figure(figsize=(10, 6))
        plt.scatter(y_val, y_pred, alpha=0.6, color="blue")
        plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], '--r', linewidth=2)
        plt.title(f"Predicted vs Actual Values - {model_name}")
        plt.xlabel("Actual Values")
        plt.ylabel("Predicted Values")
        plt.grid(True)
        plt.savefig("predicted_vs_actual.png")
        mlflow.log_artifact("predicted_vs_actual.png")
        plt.close()

        # Save the model if it's the best one found so far
        if trial.number == 0 or mse < study.best_value:
            best_model = pipeline

        return mse


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(optuna_mlflow_tuner, n_trials=50)

print("\nBest hyperparameters found:", study.best_params)

### <b>Evaluation</b>

In [ ]:
def evaluate_model(model, X, y):
    """
    Évalue un modèle de machine learning en calculant plusieurs scores d'erreur.
    
    Paramètres :
    - model : modèle entraîné (XGBoost, RandomForest, etc.)
    - X_test : Features du jeu de test
    - y_test : Valeurs réelles des entrées
    
    Retourne :
    - Un dictionnaire avec les scores MAE, RMSE et R² + un commentaire d'évaluation.
    """
    
    # Prédiction sur les données de test
    y_pred = model.predict(X)
    
    # Calcul des scores
    mae = mean_absolute_error(y, y_pred)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    r2 = r2_score(y, y_pred)


    # Retourner les résultats
    results = {
        "MAE": round(mae, 2),
        "RMSE": round(rmse, 2),
        "R²": round(r2, 2),
    }

    return results

In [ ]:
evaluate_model(best_model, X_val, y_val)

In [ ]:
def time_series_cross_validation(model, X, y, n_splits=5):
    """
    Effectue une validation croisée adaptée aux séries temporelles avec TimeSeriesSplit.
    
    Paramètres :
    - model : modèle de ML (XGBoost, RandomForest, etc.)
    - X : Features
    - y : Cible (nombre d'entrées)
    - n_splits : Nombre de splits pour la validation
    
    Retourne :
    - Moyenne des scores MAE, RMSE et R² sur les splits
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    mae_scores, rmse_scores, r2_scores = [], [], []
    
    for train_index, test_index in tscv.split(X):
        # Séparer les données en train/test
        X_retrain, X_retest = X.iloc[train_index], X.iloc[test_index]
        y_retrain, y_retest = y.iloc[train_index], y.iloc[test_index]
        
        # Entraîner le modèle
        model.fit(X_retrain, y_retrain)
        
        # Prédire sur les données de test
        y_pred = model.predict(X_retest)
        
        # Calculer les scores
        mae_scores.append(mean_absolute_error(y_retest, y_pred))
        rmse_scores.append(np.sqrt(mean_squared_error(y_retest, y_pred)))
        r2_scores.append(r2_score(y_retest, y_pred))

    # Moyenne des scores
    results = {
        "MAE moyen": round(np.mean(mae_scores), 2),
        "RMSE moyen": round(np.mean(rmse_scores), 2),
        "R² moyen": round(np.mean(r2_scores), 2),
    }
    
    return results

In [ ]:
time_series_cross_validation(best_model, X_full_train, y_full_train)

## <b>Step 7:</b> Prediction

In [ ]:
#- Retrain the model on the whole dataset 
best_model.fit(X, y)

In [ ]:
X_to_predict = [[]]
predictions = best_model.predict(X_to_predict)
print("Predictions:", predictions)

In [ ]:
data_path_2 = ".csv"
data_2 = pd.read_csv(data_path_2)
raw_data_2 = data_2.copy()
data_2.head(3)

In [ ]:
basic_inspection(data_2, visualize_nulls=True, detailed_summary=True, check_duplicates=True)

In [ ]:
X_features = ["feature1", "feature2", "feature3"]  # Replace with actual feature names
X_to_predict = data_2[X_features]
data_2 = data_cleaning(X_to_predict)

In [ ]:
predictions = best_model.predict(X_to_predict)
print("Predictions:")
print(predictions)

In [ ]:
# Add predictions to the original data
results_df = raw_data_2.copy()  # Copy the original data for context
results_df["Predicted Values"] = predictions  # Add predicted values as a new column

results_file_path = "predictions_results.csv"
results_df.to_csv(results_file_path, index=False)

print(f"Results exported successfully to {results_file_path}")